# Recomender Systems

I always look for Kaggle analysis before doing my own analysis in whatever domain. I think get some ideas before starting is very important, I also like to read the comments to get a grasp of some complication that the code could have had or things that might not be well explain in the code itself. I will add something to the Kaggle analysis or reduce certains part to make the code understandable.
The Kaggle that I was reading for this Notebook is this one:

https://www.kaggle.com/code/rounakbanik/movie-recommender-systems

Also I highly recommend these resources: 

https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD

https://sifter.org/~simon/journal/20061211.html  

In [1]:
# pip install scikit-surprise

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ast import literal_eval
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import warnings; warnings.simplefilter('ignore')

from sklearn.preprocessing import normalize


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Movie information metadata

This is the information for the movies, it gives genres, description, overall ratings and other important metrics. Here we will create a Collaborative Filtering, so this metrics we will not use it, in this notebook. But We will explore them in the future when we create a hibrid model.

In [4]:
# Read the data

X_full = pd.read_csv('movies_metadata.csv')

X_full.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [5]:
# We will use the same cleaning process as the kaggle code provided above :) 
X_full['genres'] = X_full['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
X_full['year'] = pd.to_datetime(X_full['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
X_full.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995


In [6]:
len(X_full.id.unique()) # how many movies

45436

In [7]:
str(X_full['overview'][0])

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [8]:
str(X_full['production_companies'][0])

"[{'name': 'Pixar Animation Studios', 'id': 3}]"

# Analysing Previous Kaggles and Understanding what is possible

(Here I continue to use the notebook of ROUNAK BANIK presented above)
So in the Kaggle presented there are differents way to recommend that I would like to mentioned:

- Recommend just based on populary and "general taste": this is base in recommend movies that are really like for a lot of people, let us say that those are "good movies". The columns that they use are: vote_count,	vote_average,	popularity and a new column created called wr.
here is a extract of the Kaggle:

"""

I use the TMDB Ratings to come up with our **Top Movies Chart.** I will use IMDB's *weighted rating* formula to construct my chart. Mathematically, it is represented as follows:

Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

where,
* *v* is the number of votes for the movie
* *m* is the minimum votes required to be listed in the chart
* *R* is the average rating of the movie
* *C* is the mean vote across the whole report

"""
- The other one is the content based one: The idea here is to see similarities in movies based on description and actors/directors in it. Columns to use: genres,overview,tagline. And I would add: adult and production_companies.

- Collaborative Filtering: Here we use ratings giving by other users to make predictions. This is the model we will build!!

# Collaborative Filtering

In [9]:
# Read the data

X = pd.read_csv('ratings_small.csv').sort_values(by=['movieId']).reset_index(drop=True)

X.head(3)

,userId,movieId,rating,timestamp
0,68,1,4.0,1194741818
1,261,1,1.5,1101665532
2,383,1,5.0,852806429


In [10]:
X.shape

(100004, 4)

In [11]:
len(X.movieId.unique())

9066

In [12]:
len(X.userId.unique())

671

In [13]:
reader = Reader()

In [14]:
data = Dataset.load_from_df(X[['userId', 'movieId', 'rating']], reader)

In [15]:
# We'll use the famous SVD algorithm.
svd = SVD(biased= True)

# Run 5-fold cross-validation and print results
cross_validate(svd, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8918  0.8988  0.8897  0.8974  0.9054  0.8966  0.0055  
MAE (testset)     0.6884  0.6916  0.6871  0.6911  0.6960  0.6908  0.0031  
Fit time          0.71    0.73    0.92    1.27    1.29    0.98    0.25    
Test time         0.08    0.16    0.17    0.28    0.16    0.17    0.06    


{'test_rmse': array([0.8917999 , 0.89884245, 0.88973435, 0.89740401, 0.90540752]),
 'test_mae': array([0.68837034, 0.69159241, 0.68714005, 0.69112121, 0.69600936]),
 'fit_time': (0.7128250598907471,
  0.7275521755218506,
  0.9187402725219727,
  1.2729270458221436,
  1.2922124862670898),
 'test_time': (0.07535076141357422,
  0.1579151153564453,
  0.16622066497802734,
  0.27763795852661133,
  0.16309690475463867)}

So what we got is that generally we make an error of less than 1 point in the ratings for user rating a movie that they haven't seen yet!

In [16]:
dont_print = str("""
uid – The (raw) user id. See this note.
iid – The (raw) item id. See this note.
r_ui (float) – The true rating 
est (float) – The estimated rating 
""")
svd.predict(1, 302,2) 

Prediction(uid=1, iid=302, r_ui=2, est=2.990491638708688, details={'was_impossible': False})

# What movies are similar?

Maybe this question will sound a little weird to answer with the model we created. How would we know which movies are similar if we do a collaborative filter instead of a content base model?

As it turns out the model creates two matrices one is the user matrix and the other one is the movie matrix. Let us explore this a little more..

In [17]:
# Firsly let us use the whole Dataset
data = Dataset.load_from_df(X[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()
svd = SVD(n_factors=100,biased = True)
svd.fit(trainset)

In [18]:
# movie matrix
movie_matrix_temp = svd.qi
movie_matrix = normalize(movie_matrix_temp, axis=0, norm='l1')
len(movie_matrix[8])

100

In [19]:
sum(movie_matrix[0])

0.0022739700403727847

In [20]:
# users matrix
svd.pu.shape

(671, 100)

In [21]:
# finding some popular movie to see what the model "sees" as similar
X_full['vote_count'] = X_full['vote_count'].apply(lambda x: float(x))
X_full[['id','original_title','vote_count']].sort_values(by='vote_count',ascending=False).head(10)

,id,original_title,vote_count
15480,27205,Inception,14075.0
12481,155,The Dark Knight,12269.0
14551,19995,Avatar,12114.0
17818,24428,The Avengers,12000.0
26564,293660,Deadpool,11444.0
22879,157336,Interstellar,11187.0
20051,68718,Django Unchained,10297.0
23753,118340,Guardians of the Galaxy,10014.0
2843,550,Fight Club,9678.0
18244,70160,The Hunger Games,9634.0


In [22]:
# let us see how many times each movie apears on X so that we know how many chances they got to get evaluated
count_appearance = X[['userId', 'movieId']].groupby('movieId').count().reset_index(drop=False)
count_appearance['id'] = count_appearance['movieId']
count_appearance['appearance'] = count_appearance['userId']
del count_appearance['userId']
del count_appearance['movieId']
count_appearance.head(2)

,id,appearance
0,1,247
1,2,107


Let us take the second movie: The Dark Knight

In [23]:
def get_similar_movies(index_movie,movie_matrix,X_full,X,count_appearance):
    # we have this
    movies_id = list(X.movieId.unique())
    # this are the id of the user ratings, we have to find in which position the id 155 is, is not necesarily position 154 because in can be "jump"
    tuple_index_movies = list(enumerate(movies_id))
    position_in_X = [item for item in tuple_index_movies if item[1] == index_movie][0][0]
    # interting movie to analyse
    movie = movie_matrix[position_in_X].reshape(1,-1)    
    similarities_temp = np.dot(movie,movie_matrix.T)
    similarities = list(similarities_temp.flatten())
    df_similarities_temp = pd.DataFrame({'id':movies_id,'similarities':similarities}).sort_values(by='similarities',ascending=False)
    df_similarities_temp['id']=df_similarities_temp['id'].astype(str)
    X_full['id']=X_full['id'].astype(str)
    count_appearance['id']=count_appearance['id'].astype(str)
    df_similarities = df_similarities_temp.merge(X_full[['id','title','year','genres','vote_average']] ,on='id',how='inner')
    df_similarities = df_similarities.merge(count_appearance[['id','appearance']] ,on='id',how='left')
    return df_similarities

In [24]:
df_similarities = get_similar_movies(1,movie_matrix,X_full,X,count_appearance)
df_similarities.head(20)

,id,similarities,title,year,genres,vote_average,appearance
0,592,1.323164e-06,The Conversation,1974,"[Crime, Drama, Mystery]",7.5,196
1,3114,1.309119e-06,The Searchers,1956,[Western],7.7,125
2,159,9.608834e-07,Maybe... Maybe Not,1994,"[Comedy, Drama]",6.1,8
3,364,9.073717e-07,Batman Returns,1992,"[Action, Fantasy]",6.6,200
4,480,9.059263e-07,Monsoon Wedding,2001,"[Comedy, Drama, Romance]",6.8,274
5,754,9.032211e-07,Face/Off,1997,"[Action, Crime, Science Fiction, Thriller]",6.8,3
6,593,8.870982e-07,Solaris,1972,"[Drama, Science Fiction, Adventure, Mystery]",7.7,304
7,22,7.868560e-07,Pirates of the Caribbean: The Curse of the Bla...,2003,"[Adventure, Fantasy, Action]",7.5,38
8,1721,7.836153e-07,All the Way Boys,1972,"[Adventure, Action, Comedy]",6.5,164
9,2671,7.827346e-07,Ringu,1998,"[Horror, Thriller]",6.9,64


Do these movies look similar? Well for the model they are similar even if for us are not. They are similar in a sense that similar users will rate them similarly.

Here I added more information about the movies, you can play changing the index and see if the movies that are similar in the context of collaborative filter makes sense to you :)
